**Pouya Sadeghi**

fall-1401(2022)

# AI course, project #2.2

Adversarial Search(Game)

# 01.Description

## Goals:

getting familiar with:
- player agents
- adversarial games algorithms
- $\alpha$-$\beta$ pruning

## Game:

In this game, each player chooses a line segment from the complete graph;

By creating the first triangle of the same color, the player who created it, loses.

In this implementation, we are agent `Red` and opponent is `Blue`

# 02.Initialization

In [1]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sim
from timeit import default_timer
from tqdm import tqdm

## Utils

In [3]:
def calculate_wining_chance(repeat=200, depth=2, pruning=False):
    results = {"red": 0, "blue": 0}
    game = sim.Sim(minimax_depth=depth, prune=pruning, gui=False)
    st = default_timer()
    for _ in tqdm(range(repeat)):
        results[game.play_game()] += 1
    end = default_timer()

    print(f"Results for depth={depth} {'with alpha-beta' if pruning else 'without'} pruning")
    print(f"- chance to win: {100*results['red']/repeat:.1f}%")
    print(f"- average round time: {1000*(end-st)/repeat:.3f} (ms)")
    print(f"- number of win: {results['red']}")
    print(f"- number of loss: {results['blue']}")



# 03.Player Agent:



## Algorithm:

1. MiniMax algorithm to find the best moves till specified depth.
2. Using $\alpha$-$\beta$ pruning and see the differences.

We assign an score for win/lose/draw state.

If we reached our depth-limit and a triangle wasn't formed yet, we use `heuristic` to assign a value to that state.

### heuristic:

for our heuristic(evaluate) function, we consider all valid remained moves and greedy remained moved(act as we consider sensitive situations/positions).

## How to select a child from the search tree:

By getting child-states of the current state.

### how to improve that:

We use ordering and define two class for our movements:
- greedy: try to force competitor to make a triangle and won't choose line segment between competitor's angles.
- helping: other possible choices that may not lead to our loss.

First, we try to choose from greedy moves, then we would continue with remained valid moves

# 04.Experiment

## No pruning

In [4]:
calculate_wining_chance(depth=1, pruning=False, repeat=200)

100%|██████████| 200/200 [00:00<00:00, 1075.70it/s]

Results for depth=1 without pruning
- chance to win: 98.5%
- average round time: 0.991 (ms)
- number of win: 197
- number of loss: 3


In [5]:
calculate_wining_chance(depth=3, pruning=False, repeat=200)

100%|██████████| 200/200 [00:14<00:00, 13.41it/s]

Results for depth=3 without pruning
- chance to win: 99.5%
- average round time: 74.598 (ms)
- number of win: 199
- number of loss: 1


In [6]:
calculate_wining_chance(depth=5, pruning=False, repeat=200)

100%|██████████| 200/200 [21:40<00:00,  6.50s/it]

Results for depth=5 without pruning
- chance to win: 98.5%
- average round time: 6502.485 (ms)
- number of win: 197
- number of loss: 3


## By pruning

In [7]:
calculate_wining_chance(depth=1, pruning=True, repeat=200)

100%|██████████| 200/200 [00:00<00:00, 1209.33it/s]

Results for depth=1 with alpha-beta pruning
- chance to win: 99.5%
- average round time: 0.836 (ms)
- number of win: 199
- number of loss: 1


In [8]:
calculate_wining_chance(depth=3, pruning=True, repeat=200)

100%|██████████| 200/200 [00:02<00:00, 76.22it/s]

Results for depth=3 with alpha-beta pruning
- chance to win: 99.0%
- average round time: 13.124 (ms)
- number of win: 198
- number of loss: 2


In [9]:
calculate_wining_chance(depth=5, pruning=True, repeat=200)

100%|██████████| 200/200 [00:35<00:00,  5.71it/s]

Results for depth=5 with alpha-beta pruning
- chance to win: 99.5%
- average round time: 175.149 (ms)
- number of win: 199
- number of loss: 1


In [10]:
calculate_wining_chance(depth=7, pruning=True, repeat=200)

100%|██████████| 200/200 [05:05<00:00,  1.53s/it]

Results for depth=7 with alpha-beta pruning
- chance to win: 99.0%
- average round time: 1527.218 (ms)
- number of win: 198
- number of loss: 2


# 05.Discussion

## 01. what are the features of a good heuristic? why did choose this heuristic?

Features of a good heuristic:
- a good heuristic should be able to predict close to reality(guess the win or loss ahead of time).
- able to identify states that provide a better situation for us(assign higher score to them).


The winning aspect of our heuristic:
- considering number of valid move remained for us(try to postpone the loss, if we don't have winning strategy).
- in addition to paying attention to having moves to play for the future, tries to make things difficult for opponent.


## 02. What is the effect of the depth of the algorithm on the chance of winning, time and explored states?

On the chance of winning:
- increases the chance of winning, due to strategy determination for more number of moves.
- we expect to see winning chance improved by higher depth, but sometimes we may see a different behavior due to randomized opponent(not optimal).

On the execution time:
- increases the execution time because of having more states to explore.

On the explored stated:
- $O(b^d)$ is the order of explored state (*b* is branching factor and *d* is depth); so, by increasing depth, number of explored states would increase.

## 03. When we use pruning, what is the order of exploring the children of each state? Does this order matter? Why is this arrangement used?

As mentioned before, we defined two class of children for each state: *greedy* and *helping*;
- greedy: they care about potential opponent's triangles and keep the opponents's chance of creating a triangle(try to decrease opponent valid moves).
- helping: rest of valid moves.

We explore helping states after we explored greedy states.

We expect that greedy states would be a better choice for us(in case of score);
also, in $\alpha$-$\beta$ pruning, we define a threshold based on states that explored before, to decide whether to explore the next states or not.

In this way, we try to find the best child-state sooner and skip(prune) more children(state).

This ordering plays important role to making our agent with enabled $\alpha$-$\beta$ pruning faster;
but it doesn't play an important role is speed when it is disabled.